In [ ]:
# Install dependencies for this example
# Note: This does not include itkwidgets, itself
import sys
!{sys.executable} -m pip install --upgrade itk-io itk-cuberille itk-totalvariation

In [1]:
from urllib.request import urlretrieve
import os

import numpy as np
import itk

from itkwidgets import view
import itkwidgets

In [2]:
# Download data
# Source: https://data.broadinstitute.org/bbbc/BBBC024/
file_name = 'HL50_cell_line_c00_03_extraction.tif'
if not os.path.exists(file_name):
    url = 'https://data.kitware.com/api/v1/file/5b61f16c8d777f06857c1949/download'
    urlretrieve(url, file_name)

In [6]:
image = itk.imread(file_name, itk.F)

view(image, cmap=itkwidgets.cm.BrBG, annotations=False, vmax=800, ui_collapsed=True)

Viewer(annotations=False, cmap='BrBG', geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<it…

In [7]:
# Segment the cells
smoothed = itk.prox_tv_image_filter(image, weights=50, maximum_number_of_iterations=3)

LabelImageType = itk.Image[itk.UC, 3]

threshold_filter = itk.MomentsThresholdImageFilter.New(smoothed)
threshold_filter.Update()
threshold = threshold_filter.GetThreshold()

In [8]:
interpolator = itk.BSplineInterpolateImageFunction.New(smoothed, spline_order=3)
mesh = itk.cuberille_image_to_mesh_filter(smoothed,
                                          interpolator=interpolator,
                                          iso_surface_value=threshold,
                                          project_vertices_to_iso_surface=True,
                                          project_vertex_surface_distance_threshold=0.5)
view(geometries=mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…